### According to the entered data provides all the information about a person's consumption

In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

month = str(datetime.now().month)
if len(month)==1:
    month='0'+month

year = str(datetime.now().year)
y = year[2::]  
y1 = str(int(y)+1)
    
def get_query(name1, surname1, midname1, tel, card1):
    query = f"""
    select distinct phone_num, email_addr 
    from
    public.clients_info 
    where """
    if name1 != '':
        query += f"fst_name='{name1}' "
    if surname1 != '' and name1 == '':
        query += f"last_name='{surname1}' "
    elif surname1 != '' and name1 != '':
            query += f"and last_name='{surname1}' "
    if midname1 != '' and name1 == '' and surname1 == '':
        query += f"mid_name = '{midname1}' "
    elif midname1 != '' and (name1 != '' or surname1 != ''):
        query += f"and mid_name = '{midname1}' "
    if tel != '' and surname1 == '' and midname1 == '' and name1 == '':
        query += f" phone_num= '+7{tel}' "
    elif tel != '' and (name1 != '' or surname1 != '' or midname1 != ''):
        query += f"and phone_num= '+7{tel}' "
    if card1 != '' and tel == '' and surname1 == '' and midname1 == '' and name1 == '':
        query += f" card_no='{card1}' "
    elif card1 != '' and (name1 != '' or surname1 != '' or midname1 != '' or tel != ''):
        query += f"and card_no='{card1}' "
    query +=  f""" group by phone_num, email_addr 
    """
    return query

def query (tel,email,month,y):
    query = f"""
    select distinct card_no, fst_name, last_name, mid_name, sex_mf, date_part('year', age(now(),clients_info.birth_dt)) as age, phone_num, email_addr, public.cls_azs.state as state, round(sum(ta_ntu),2) as ta_ntu, round(sum(ta_fuel),2) as ta_fuel, round(sum(cheque_id)) as cheques, sum(ltr_vol) as volume, sum(qnt) as quantity
    from 
    (select card_no, count(cheque_id) as cheque_id, qnt, ltr_vol, ta_ntu, ta_fuel, org_ksss
    from
    public.pet_chk_{y}
    group by card_no, qnt, ltr_vol, ta_ntu, ta_fuel, org_ksss
    union all
    select discount_card_info as card_no, count(cheque_id) as cheque_id, sum(quantity) as qnt, sum(volume) as ltr_vol, sum(case when quantity is not null then amount_payment else 0 end) as ta_ntu, sum(case when volume is not null then amount_payment else 0 end) as ta_fuel, org_ksss                                                                        
    from pet.pet_{month}{y}
    group by discount_card_info, org_ksss) a
    right join 
    public.clients_info using(card_no) 
    left join public.cls_azs on a.org_ksss=public.cls_azs.ksss_union
    where  """
    if tel != '':
        query += f" phone_num='{tel}' "
    elif tel == '':
        query += f" phone_num is null "
    if email != '':
        query += f" and email_addr='{email}' "
    query += f"""
    group by card_no, fst_name, last_name, mid_name, sex_mf, birth_dt, phone_num, email_addr, public.cls_azs.state 
    order by state
    """
    return query

def query_telnone_emailnone (name1, surname1, midname1, card1, month, y):
    query = f"""
    select distinct card_no, fst_name, last_name, mid_name, sex_mf, date_part('year', age(now(),clients_info.birth_dt)) as age, phone_num, email_addr, public.cls_azs.state as state, round(sum(ta_ntu),2) as ta_ntu, round(sum(ta_fuel),2) as ta_fuel, round(sum(cheque_id)) as cheques, sum(ltr_vol) as volume, sum(qnt) as quantity
    from 
     (select card_no, count(cheque_id) as cheque_id, qnt, ltr_vol, ta_ntu, ta_fuel, org_ksss
    from
    public.pet_chk_{y}
    group by card_no, qnt, ltr_vol, ta_ntu, ta_fuel, org_ksss
    union all
    select discount_card_info as card_no, count(cheque_id) as cheque_id, sum(quantity) as qnt, sum(volume) as ltr_vol, sum(case when quantity is not null then amount_payment else 0 end) as ta_ntu, sum(case when volume is not null then amount_payment else 0 end) as ta_fuel, org_ksss                                                                        
    from pet.pet_{month}{y}
    group by discount_card_info, org_ksss) a
    right join 
    public.clients_info using(card_no) 
    left join public.cls_azs on a.org_ksss=public.cls_azs.ksss_union
    where  """
    if name1 != '':
        query += f"fst_name='{name1}' "
    if surname1 != '' and name1 == '':
        query += f"last_name='{surname1}' "
    elif surname1 != '' and name1 != '':
            query += f"and last_name='{surname1}' "
    if midname1 != '' and name1 == '' and surname1 == '':
        query += f"mid_name = '{midname1}' "
    elif midname1 != '' and (name1 != '' or surname1 != ''):
        query += f"and mid_name = '{midname1}' "
    if card1 != '' and surname1 == '' and midname1 == '' and name1 == '':
        query += f" card_no='{card1}' "
    elif card1 != '' and (name1 != '' or surname1 != '' or midname1 != ''):
        query += f"and card_no='{card1}' "
    query += f""" and phone_num is null and email_addr is null
    group by card_no, fst_name, last_name, mid_name, sex_mf, birth_dt, phone_num, email_addr, public.cls_azs.state 
    order by state
    """
    return query

def get_points_obtaining(i, y):
    return f"""
    select distinct card_no, sum(earning_rub)/100 as points_obtaining
    from public.actions_crm_{y}
    group by card_no
    having card_no='{i}'
    """

def get_nonpoints(i, month, y):
    return f"""
    select card_no, nonpoints_discount, round( cast (nonpoints_discount/amount*100 as numeric),2)as discount
    from 
    (select card_no, sum(amount) as amount
    from
    (select card_no, sum(amount) as amount
    from public.pet_chk_{y}
    group by card_no
    union all
    select discount_card_info as card_no, sum(amount_goods) as amount
    from pet.pet_{month}{y}
    group by discount_card_info) as d
    group by card_no) as c 
    inner join
    (select
    a.discount_card_info as card_no, sum(a.promo_discount) as nonpoints_discount
    from 
    (select distinct *
    from public.actions_petronix_{y}
    where date_part('year',operation_date)='20{y}' and discount_card_info = '{i}') a
    group by a.discount_card_info) b 
    using (card_no)
    """

def get_points_and_coupons_usage(i, month, y):
    return f"""
    select card_no, sum(points) as points_discount, round(sum(points)/a.amount*100,2) as discount
    from 
    (select card_no, round(sum(amount),2) as amount
    from
    (select card_no, sum(amount) as amount
    from public.pet_chk_{y}
    group by card_no
    union 
    select discount_card_info as card_no, sum(amount_goods) as amount
    from pet.pet_{month}{y}
    group by discount_card_info) as b
    group by card_no) as a
    inner join   
    (select card_no, cheque_id, operation_date,
    case when raz.r<1 then 0
    else raz.r
    end as points
    from (select discount_card_info as card_no, cheque_id, operation_date, floor(amount_goods-amount_payment) as r
    from pet.pet_{month}{y})  as raz
    where cheque_id not in (select cheque_id from public.actions_petronix_{y})
    union all
    select card_no, cheque_id, a.operation_date, floor(sum(amount - discount)) as points
    from 
    (select distinct discount_card_info as card_no, cheque_id, operation_date, sum(amount_goods-amount_payment) as amount
    from pet.pet_{month}{y}
    group by card_no, cheque_id, operation_date) a 
    inner join 
    (select distinct discount_card_info, cheque_id, operation_date, cast(promo_discount as numeric) as discount
    from public.actions_petronix_{y}
    where date_part('year',operation_date)='20{y}') c using(cheque_id)
    group by card_no, cheque_id, a.operation_date
    union all 
    select * from public.points_usage_m) as p using (card_no)
    group by card_no, amount
    having card_no='{i}'
    """

def get_count_coupons(i, y, y1):    
    return f"""
    select card_no, count(coupon_id) as coupons 
    from public.payment_coupons 
    where redeemed_at is not null and assign_date is not null and '20{y}-01-01' < redeemed_at and redeemed_at < '20{y1}-01-01'
    group by card_no
    having card_no='{i}'
    """
def get_azs (i, month, y):
    return f"""
    select card_no, org_name, sum(cheqamount) as cheqamount
    from
    (select card_no, org_name, count(cheque_id) as cheqamount
    from 
    public.pet_chk_{y}
    group by card_no, org_name
    union all
    select discount_card_info as card_no, org_name, count(cheque_id) as cheqamount
    from
    pet.pet_{month}{y}
    group by discount_card_info, org_name) as a
    group by card_no, org_name
    having card_no = '{i}'
    """

def get_fuel_types(i, month, y):
    query = f"""
select 
    card_no, 
    round(sum(ai92),2) as ai92, 
    round(sum(ai95),2) as ai95, 
    round(sum(ai98),2) as ai98,
    round(sum(diesel),2) as diesel,
    round(sum(gaz),2) as gaz,
    round(sum(other),2) as other
from 
(
select 
    card_no, 
    sum(case when vol_ai92>1 then vol_ai92 else 0 end) as ai92,
    sum(case when vol_ai95>1 then vol_ai95 else 0 end) as ai95,
    sum(case when vol_ai98>1 then vol_ai98 else 0 end) as ai98,
    sum(case when vol_diesel>1 then vol_diesel else 0 end) as diesel,
    sum(case when vol_gaz>1 then vol_gaz else 0 end) as gaz,
    sum(case when vol_other>1 then vol_other else 0 end) as other
from 
public.pet_chk_{y}
group by card_no

union all
    
select 
    distinct discount_card_info as card_no, 
    sum(case when type2='АИ-92' then ltr else 0 end) as ai92, 
    sum(case when type2='АИ-95' then ltr else 0 end) as ai95,
    sum(case when type2='АИ-100/98' or type2='АИ-100' then ltr else 0 end) as ai98,
    sum(case when type2='ДТ' then ltr else 0 end) as diesel,
    sum(case when type2='СУГ' then ltr else 0 end) as gaz,
    sum(case when type2='ПРОЧ' then ltr else 0 end) as other
from(
select discount_card_info, type2, sum(ltr) as ltr
from
((select discount_card_info, gds_ksss, sum(volume) as ltr
from pet.pet_{month}{y}
group by discount_card_info,  gds_ksss) as pet
inner join 
(select gds_ksss, type2
from public.cls_global) as fuel using(gds_ksss))
group by discount_card_info, type2) as a
group by discount_card_info
) as main 
group by card_no
having card_no='{i}'
"""
    return query

def test1 (x):
    if x == 0:
        return 'Не указано'
    else:
        
        return x.title()
    
def test (x):
    if x == 0:
        return 'не указано'
    else:
        return x
    

name1 = input('Укажите имя: ').upper()
surname1 = input('Укажите фамилию: ').upper()
midname1 = input('Укажите отчество: ').upper()
tel = input('Укажите номер телефона: +7')
card1 = input('Укажите номер карты: ')

print('Спасибо, запрос обрабатывается...\nСреднее время обработки запроса: 2 минуты')

df1 = pd.read_sql(get_query(name1, surname1, midname1, tel, card1), connection)


# Создаю лист номеров телефона и имейлов
phone = df1.phone_num.to_list()
email = df1.email_addr.to_list()

if len(phone)==0:
    print('Информация не найдена')
else:
    print(f'''
            Найдено клиентов в базе: {len(phone)}
                    ''')
    print('==========================================================================')

# Создаю словарь из номеров телефонов и имейлов для поиска NONE в телефонах
m = dict()
for a,b in zip (phone, email):
    if a in m.keys():
        if b not in m.values():
            m[a]+= b
        else:
            continue  
    else:
        m[a]=b    

counter = 0
# Создание таблиц по каждому человеку
for key, value in m.items():
    if key == None and value != None:
        tel = ''
        email = value
        df = pd.read_sql(query (tel,email, month, y), connection)
    elif key == None and value == None:
        df = pd.read_sql(query_telnone_emailnone (name1, surname1, midname1, card1, month, y), connection)
    else:
        tel = key
        email = ''
        df = pd.read_sql(query (tel,email, month, y), connection)
    
    df.fillna(0, inplace=True)
    counter+=1
# Считывание инфы из таблицы       
    card_no1 = df.card_no.unique()
    card_no = (', \n'.join(map(str, (df.card_no.unique()).ravel())))
    fst_name = (', \n'.join(map(str, (df.fst_name.unique()).ravel())))        
    last_name = (', \n'.join(map(str, (df.last_name.unique()).ravel())))
    mid_name1 = df.mid_name.unique() 
    sex1 = df.sex_mf.unique()
    age1 = df.age.unique()
    email1 = df.email_addr.unique()
    state = (', \n'.join(map(str, (df[~df['state'].isna()]['state'].unique()).ravel())))
    phone1 = df.phone_num.unique()
    

# Замена None на "не указано" в некоторых параметрах (отчество, пол, возраст, имейл)
    mid_name = list(map(test1, mid_name1))            
    sex =  list(map(test, sex1))
    age = list(map(test, age1))
    email = list(map(test, email1))
    phone = list(map(test, phone1))
    

# Считывание % чеков (частота) для областей и создание листа этих значений
    cheques = df.cheques
    cheq = []
    for i in cheques:
        j = int(i)
        j = ((j/df.cheques.sum())*100).round(2)
        cheq+=[j]
    
# Создание листа областей и карт
    states = list(df.state)
    card = list (df.card_no)
    c = dict()
    notactive = 0
    active = 0
    
# Соединение листа областей и частоты (по кол-ву чеков) в словарь d
    d = dict()
    for a,b in zip (states, cheq):
        if a in d.keys():
            d[a]+= b
        else:
            d[a] = b
            
# Удаление областей None из словаря d
    for key in list(d.keys()):
        if key == 0:
            del d[key]
        else:
            continue  
            
# Соединение листа карт и областей в словарь с
    for a,b in zip (card, states):
        if a in c.keys():
            if a == 0:
                a=0
                c[a]+= b     
        else:
            c[a]= b

# Использование словаря с для подсчёта кол-ва активных и неактивных карт
    for value in c.values():
        if value == 0:
            notactive+=1
        else:
            active+=1
    
    
# Подсчёт сум чеков, кол-ва литров, кол-ва товаров, рублей на литры, рублей на нту
    all_cheques = df.cheques.sum()
    volume = df.volume.sum()
    quantity = df.quantity.sum()
    fuel = df.ta_fuel.sum()
    ntu = df.ta_ntu.sum()
    

# Вывод основной инфы
    print(f'''                    Клиент №{counter}

ФИО: {last_name.title()} {fst_name.title()} {', '.join(str(i) for i in mid_name)}
Пол: {', '.join(str(i) for i in sex)}
Возраст: {', '.join(str(i) for i in age)}
Номер телефона: {', '.join(str(i) for i in phone)}
Адрес электронной почты: {', '.join(str(i) for i in email)}
Потребление ({year}): {volume:.0f} литров ({fuel:.0f}руб.), {quantity :.0f} товаров ({ntu:.0f}руб.), ({all_cheques} чеков)

Номер карты ({len(df.card_no.unique())}): {card_no}

Количество активных карт: {active}    Количество неактивных карт: {notactive}
''')
    if len(d) == 0:
        print('Регионы заправки: 0')
    else:
        print('Регионы заправки:')
        for key, value in d.items():
            print('{}: {}%'. format(key, value))

    # Вывод инфы по каждой отдельной карте
    for i in card_no1:
        # Получение таблицы с начисленными баллами по бальным акциям
        df = pd.read_sql(get_points_obtaining(i,y), connection)
        points = df.points_obtaining.unique()
        if len(points) == 0:
            points=[0]
        print(f'''
                    Номер карты: {i}''')
        print(f'''Начисленно баллов по бальным акциям: {', '.join(str(i) for i in points)} ''')

        # Получение таблицы с полученной скидкой по безбальным акциям
        df = pd.read_sql(get_nonpoints(i,month,y), connection)
        nonpoints = df.nonpoints_discount
        ndiscount = df.discount
        if len(nonpoints)==0:
            nonpoints=[0]
        print(f"""Полученная скидка по безбальным акциям: {', '.join(str(i) for i in nonpoints)} ({', '.join(str(i) for i in ndiscount)}%)  """)
        
        
        # Получение таблицы с полученной скидкой за счёт использования купонов и списания баллов
        df = pd.read_sql(get_points_and_coupons_usage(i, month, y), connection)
        points = df.points_discount
        discount = df.discount
        if len(points)==0:
            points=[0]
        print(f"""Полученная скидка за счёт использования купонов и списания баллов: {', '.join(str(i) for i in points)} ({', '.join(str(i) for i in discount)}%)  """)

        # Получение таблицы с кол-вом использованных купонов
        df = pd.read_sql(get_count_coupons(i,y,y1), connection)
        coupons = (', \n'.join(map(str, (df.coupons.unique()).ravel())))
        if len(coupons)==0:
            coupons = 0
        print(f'''Использовано купонов: {coupons} ''')
        
        # Получение таблицы с номерами АЗС и кол-вом чеков
        df = pd.read_sql(get_azs (i, month,y), connection)
        cheqamount = df.cheqamount
        org_name = df.org_name
        
        
        # Считывание % чеков (частота) для АЗС и создание листа этих значений
        cheqam = df.cheqamount
        cheqpercent = []
        for k in cheqam:
            k = int(k)
            k = ((k/df.cheqamount.sum())*100).round(2)
            cheqpercent+=['('+str(k)+'%'+')']
            
        # Таблица с АЗС, кол-во чеков и % 
        if len(org_name)>0:
            df2 = pd.DataFrame({'АЗС:':org_name, ' ':cheqamount.round(), '':cheqpercent}). set_index('АЗС:')
            print(df2)
        else:
            print(f'''
АЗС: 0''')
        
        # Получение таблицы с типпом топливо и количеством приобретённого топлива
        df = pd.read_sql(get_fuel_types(i, month,y), connection)
        ai92 = (', \n'.join(map(str, (df.ai92.unique()).ravel())))
        ai95 = (', \n'.join(map(str, (df.ai95.unique()).ravel())))
        ai98 = (', \n'.join(map(str, (df.ai98.unique()).ravel())))
        diesel = (', \n'.join(map(str, (df.diesel.unique()).ravel())))
        gaz = (', \n'.join(map(str, (df.gaz.unique()).ravel())))
        other = (', \n'.join(map(str, (df.other.unique()).ravel())))
        if len(ai92)==0:
            ai92=0.0
        if len(ai95)==0:
            ai95=0.0
        if len(ai98)==0:
            ai98=0.0
        if len(diesel)==0:
            diesel=0.0
        if len(gaz)==0:
            gaz=0
        if len(other)==0:
            other=0.0
        print(f''' 
АИ-92: {ai92} литров
АИ-95: {ai95} литров
АИ-98: {ai98} литров
Дизель: {diesel} литров
Газ: {gaz} литров
Прочие: {other} литров
        ''')
    print('==========================================================================')

if len(phone)==0:
    print('')
else:
    print('Вся найденная информация выведена')
    